# Titanic - Machine Learning from Disaster 

## About Dataset

<img src="https://imgur.com/rE1OxtK.jpg">

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

The goal in this project is to use the Titanic passenger data (name, age, gender, socio-economic class, etc) to predict weather or not passenger survived.

## Data Source

The dataset has been referred from Kaggle and it can be downloaded [here](https://www.kaggle.com/c/titanic/data).